In [1]:
!pip3 install GitPython langchain langchain-community langchain_openai pip install python-dotenv faiss-cpu  -qU

In [2]:
from git import Repo

repo = Repo.clone_from("https://github.com/langchain-ai/langserve", "./langserve_repo")
branch = repo.head.reference


In [3]:
from langchain.document_loaders import GitLoader
import os

In [4]:
loader = GitLoader(repo_path="./langserve_repo/", branch=branch)


In [5]:
langserve_data = loader.load()


In [6]:
len(langserve_data)


216

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=100,
)

langserve_data = splitter.split_documents(langserve_data)


In [8]:
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [9]:
from langchain_openai import OpenAIEmbeddings

emb_model = OpenAIEmbeddings()

In [10]:
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(langserve_data, embedding=emb_model)


In [12]:
db.save_local("./langserve_index")